# **Ejemplo de Aplicación del Método Desacoplado Rápido para la Solución de Flujos de Potencia**


Inicialmente, se toman los datos del sistema de potencia: matriz de admitancias, voltajes y potencias iniciales.

In [1]:
import numpy as np

# Definir datos del sistema
n=3
Ybus=np.array([(15-35j,-10+20j,-5+15j),(-10+20j,30-60j,-20+40j),(-5+15j,-20+40j,25-55j)],dtype=complex)
B=np.imag(Ybus)
G=np.real(Ybus)

# Valores definidos de potencia
Pdef=np.array([[0.5,-1]])
Qdef=np.array([[-0.6]])

# Definir valores iniciales de voltaje
def rad(deg):
    return deg*np.pi/180

vbusMag=np.array([1.02,1.02,1.02])
vbusAng=np.array([rad(0),rad(0),rad(0)])

def asvoltage(Mag,Ang):
  # Mag: vector de magnitudes de voltaje
  # Ang: vector de ángulos de voltaje
  v=np.zeros((len(Mag),1),dtype=complex)
  for i in range(len(Mag)):
    v[i]=Mag[i]*(np.cos(Ang[i])+1j*np.sin(Ang[i]))
  return v
vbus=asvoltage(vbusMag,vbusAng)
print('Voltajes:\n',vbus)

# Calcular potencias inyectadas a cada bus
S=np.zeros((n,1),dtype=complex)
for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))

print('Potencia compleja:\n',S)

# Calcular residuos
def residuos(S,Pdef,Qdef):
  Qcal=np.imag(S)
  Pcal=np.real(S)
  dP=np.swapaxes(Pdef,0,1)-np.real(S)[1:3]
  dQ=Qdef-np.imag(S)[2]
  return dP, dQ

[dP,dQ]=residuos(S,Pdef,Qdef)
print('Residuos:',dP,dQ)

Voltajes:
 [[1.02+0.j]
 [1.02+0.j]
 [1.02+0.j]]
Potencia compleja:
 [[1.81188398e-15+3.62376795e-15j]
 [3.62376795e-15+7.24753590e-15j]
 [0.00000000e+00+7.24753590e-15j]]
Residuos: [[ 0.5]
 [-1. ]] [[-0.6]]


Como los residuos son mayores a la tolerancia (supongamos que es 10^-6), s edebe continuar a calcular el Jacobiano y actualizar las variables.

In [2]:
count=0
pconocida=[2,3]
qconocida=[3]
Qcal=np.imag(S)
Pcal=np.real(S)

B1=np.zeros((len(pconocida),len(pconocida)))
for i in range(len(pconocida)):
  for j in range(len(pconocida)):
    k=pconocida[i]-1
    m=pconocida[j]-1
    B1[i][j]=-np.imag(Ybus[k][m])

B2=np.zeros((len(qconocida),len(qconocida)))
for i in range(len(qconocida)):
  for j in range(len(qconocida)):
    k=qconocida[i]-1
    m=qconocida[j]-1
    B2[i][j]=-np.imag(Ybus[k][m])

dPQ=np.concatenate((dP,dQ),axis=0)

while ((max(abs(dPQ))>10e-4) & (count<20)):
  # Calcular variaciones
  dD=np.linalg.solve(B1, dP)
  dV=np.linalg.solve(B2,dQ)

  # Actualizar voltajes
  for i in range(n-1):
    vbusAng[pconocida[i]-1]+=dD[i]
  for i in range(len(qconocida)):
    vbusMag[qconocida[i]-1]=vbusMag[qconocida[i]-1]*(1+dV[i])
  vbus=asvoltage(vbusMag,vbusAng)
  print('Voltajes: \n',vbusMag)
  print('Ángulos:\n',vbusAng)

  #Actualizar potencias
  for i in range(0,n):
    S[i]=vbus[i]*np.conjugate(np.matmul(Ybus[i,:],vbus))
  Qcal=np.imag(S)
  Pcal=np.real(S)
  print('Potencias:',S)

  #Calular residuos
  [dP,dQ]=residuos(S,Pdef,Qdef)
  dPQ=np.concatenate((dP,dQ),axis=0)
  print('Residuos: \n',dP,dQ)

  count+=1
  print('Iteración:',count)

Voltajes: 
 [1.02       1.02       1.00887273]
Ángulos:
 [ 0.         -0.00735294 -0.02352941]
Potencias: [[ 0.57461386-0.02247042j]
 [ 0.74279869+0.20352654j]
 [-1.30552071-0.15380491j]]
Residuos: 
 [[-0.24279869]
 [ 0.30552071]] [[-0.44619509]]
Iteración: 1
Voltajes: 
 [1.02       1.02       1.00068811]
Ángulos:
 [ 0.         -0.00801947 -0.01845924]
Potencias: [[ 0.54916665+0.12111423j]
 [ 0.65477011+0.6611389j ]
 [-1.18998012-0.75073616j]]
Residuos: 
 [[-0.15477011]
 [ 0.18998012]] [[0.15073616]]
Iteración: 2
Voltajes: 
 [1.02       1.02       1.00343065]
Ángulos:
 [ 0.         -0.00855662 -0.01539571]
Potencias: [[ 0.49988805+0.08828588j]
 [ 0.44082028+0.62677561j]
 [-0.93091261-0.69288436j]]
Residuos: 
 [[ 0.05917972]
 [-0.06908739]] [[0.09288436]]
Iteración: 3
Voltajes: 
 [1.02       1.02       1.00512525]
Ángulos:
 [ 0.         -0.00826757 -0.01644162]
Potencias: [[ 0.5017739 +0.06008033j]
 [ 0.46766291+0.52738214j]
 [-0.96043851-0.56697387j]]
Residuos: 
 [[ 0.03233709]
 [-0.03